## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-04-20-52-35 +0000,nypost,Man shoots armed peeping Tom gawking at sister...,https://nypost.com/2025/08/04/us-news/man-shoo...
1,2025-08-04-20-48-32 +0000,nypost,Married mom and English teacher pleads to 21 s...,https://nypost.com/2025/08/04/us-news/married-...
2,2025-08-04-20-46-00 +0000,wsj,Dow Bounces Back From Last Week’s Slump,https://www.wsj.com/finance/stocks/customurl-g...
3,2025-08-04-20-44-00 +0000,wsj,Stock Market Today: Dow Gains Nearly 600 Point...,https://www.wsj.com/livecoverage/stock-market-...
4,2025-08-04-20-43-57 +0000,wapo,"At least 68 killed, more than 70 feared dead a...",https://www.washingtonpost.com/world/2025/08/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
85,trump,42
159,new,21
312,gaza,13
35,dead,10
47,will,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
224,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...,86
155,2025-08-04-13-01-22 +0000,bbc,Hundreds of Israeli ex-officials appeal to Tru...,https://www.bbc.com/news/articles/crkznje8nz8o...,85
213,2025-08-04-04-46-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...,83
166,2025-08-04-11-53-00 +0000,wsj,Eric and Donald Trump Jr. to Own Millions of S...,https://www.wsj.com/finance/investing/eric-and...,78
167,2025-08-04-11-47-24 +0000,nyt,Trump to Appoint New Top Labor Statistics Offi...,https://www.nytimes.com/2025/08/03/us/politics...,77


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
224,86,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...
189,54,2025-08-04-09-03-26 +0000,nyt,Texas Redistricting Fight Prompts Democratic G...,https://www.nytimes.com/2025/08/04/us/politics...
66,43,2025-08-04-18-01-19 +0000,nypost,Stricken mom of Israeli hostage Rom Braslavski...,https://nypost.com/2025/08/04/world-news/new-c...
72,42,2025-08-04-17-48-14 +0000,nypost,Israel is ‘on the precipice of defeat’ in Gaza...,https://nypost.com/2025/08/04/world-news/israe...
121,40,2025-08-04-15-16-00 +0000,wsj,Elon Musk Gets $23.7 Billion Stock Award From ...,https://www.wsj.com/business/autos/tesla-appro...
74,38,2025-08-04-17-36-12 +0000,nypost,Boeing fighter jet workers go on strike after ...,https://nypost.com/2025/08/04/business/boeing-...
195,36,2025-08-04-08-00-20 +0000,cbc,The nuclear arms race is heating up. Will it t...,https://www.cbc.ca/radio/sunday/hiroshima-naga...
4,32,2025-08-04-20-43-57 +0000,wapo,"At least 68 killed, more than 70 feared dead a...",https://www.washingtonpost.com/world/2025/08/0...
33,32,2025-08-04-19-31-32 +0000,nypost,Trump lambasts ‘Russian war machine’ as specia...,https://nypost.com/2025/08/04/world-news/trump...
47,30,2025-08-04-19-06-46 +0000,nypost,Far left Sen. Warren gushes over NYC mayoral c...,https://nypost.com/2025/08/04/us-news/far-left...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
